<a href="https://colab.research.google.com/github/gagip/Starcraft2_Behavioral-analytics/blob/master/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 Import

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd # 메모리 제한을 막기 위해 가상 데이터 프레임 사용

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train = dd.read_csv("/content/drive/My Drive/데이터과학/프로젝트/게임행동/data/train.csv")
train.head()

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"


# 데이터 요약

In [10]:
df_train = pd.DataFrame(columns=['game_id'])
df_train.game_id = train.game_id.unique().compute()

df_train.head()

,game_id
0,0
1,1
2,2
3,3
4,4


# 게임 시간 변수

In [13]:
df_train['time'] = np.array(train[train.shift(-1).game_id != train.game_id].time.compute())
df_train.head()

,game_id,time
0,0,7.24
1,1,9.59
2,2,9.59
3,3,9.59
4,4,5.43


# 종족 열 추가

In [17]:
df = train[train.player == 0]
df_train['p0_species'] = np.array(df[df.shift(-1).game_id != df.game_id].species.compute())
df = train[train.player == 1]
df_train['p1_species'] = np.array(df[df.shift(-1).game_id != df.game_id].species.compute())

df_train.head()

,game_id,time,p0_species,p1_species
0,0,7.24,T,T
1,1,9.59,P,T
2,2,9.59,P,Z
3,3,9.59,T,P
4,4,5.43,T,Z


# 이벤트 카운트 추가

In [18]:
df = train[train.player==0]
df_train['event_count_0'] = np.array(df.game_id.value_counts()[df.game_id.unique()].compute())
df = train[train.player==1]
df_train['event_count_1'] = np.array(df.game_id.value_counts()[df.game_id.unique()].compute())

df_train.head()

,game_id,time,p0_species,p1_species,event_count_0,event_count_1
0,0,7.24,T,T,592,548
1,1,9.59,P,T,1223,1211
2,2,9.59,P,Z,851,1371
3,3,9.59,T,P,1389,1622
4,4,5.43,T,Z,966,651


# 이벤트별 카운트 열 추가
이벤트 없을 시 결측치가 발생하여 데이터프레임 병합 후 결측치 0으로 채움

In [0]:
for event in train.event.unique():
    df = train[(train.player==0)&(train.event==event)]
    df = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event+'_count_0'}, axis = 1)
    df['game_id']= np.array(df.index)
    df_train = pd.merge(df_train, df, on='game_id', how='left')

    df = train[(train.player==1)&(train.event==event)]
    df = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event+'_count_1'}, axis = 1)
    df['game_id']= np.array(df.index)
    df_train = pd.merge(df_train, df, on='game_id', how='left')
df_train = df_train.fillna(0)